
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 2 - Minimax Algorithm<br>
<font color=696880 size=4>
Sepehr Harfi


____

# Personal Data

In [1]:
import numpy as np

# Set your student number and name
Student_number = "401106266"
Name = "Mahdi"
Last_name = "Ali nejad"

# Rules

<font color=red>
Please run all the cells.

Please refrain from making any changes to the existing files, such as `board.py`, `game.py`, etc. 
</font>

#### **Note:** It is strongly recommended to execute this notebook on your local device instead of Google Colab due to limitations with graphics. However, if you still prefer using Google Colab, you can refer to this [link](https://vishnudsharma.medium.com/visualizing-tkinter-and-pygame-in-colab-272c5a245f8c) for assistance in using graphics with Colab.

# Libraries & Imports

**Dont change the below cell.**

In [2]:
import os

curr_working_directory = os.getcwd()
src_directory_path = os.path.join(curr_working_directory, "src")

if os.path.exists(src_directory_path):
    os.chdir(src_directory_path)

In [3]:
import game
from player import Player
from random_player import RandomPlayer
from random_greedy_player import RandomGreedyPlayer
import time


# Breakthrough Game: Minimax and AlphaBeta Players (100 Points)

In this notebook, we will implement and compare two AI strategies, Minimax and AlphaBeta, for playing a simple version of the Breakthrough game. We aim to assess the performance of these strategies in terms of their execution time and win probability in matches against a Greedy player and against each other with varying depths.


# Breakthrough Game Rules

The Breakthrough game is a two-player strategy board game played on an 6x6 grid. The objective of the game is to move your pieces from one end of the board to the other before your opponent does.

## Game Setup

- The board consists of a 6x6 grid with alternating dark and light squares.
- Each player starts with 12 pieces placed on their respective sides of the board.
- The pieces are initially arranged in two rows, with each row containing 6 pieces.

## Piece Movement

- Each player can only move their own pieces.
- Pieces can move diagonally or straight forward one square at a time.
- Pieces cannot move backward or sideways.
- Pieces can capture their opponent's pieces by moving diagonally forward and landing on a square occupied by an opponent's piece.
- Captured pieces are removed from the board.


## Game End

- If a player successfully moves one of their pieces to the opposite end of the board, they win the game.

## Additional Rules

- Players can only capture the opponent's piece if its in their diagonal forward squares and they cannot have two or more pieces of their color in the same square.
- Players take turns moving their pieces.
- Players must make a move on their turn, and they cannot skip their turn.
- If a player has multiple legal moves, they can choose which piece to move.


## Demo of the game with graphics

Initially, using the cell below, you can see a demo gameplay of two Random Greedy Players to gain insights into the game. Additionally, you can explore the gameplay of other agents such as "Player" which plays in a complete greedy way and also "RandomPlayer", to further enhance your understanding of the game.

**Note:** You can use the cell below anywhere you want to see the game with the graphics. For this purpose, you should only be careful to pass the right class of players to the `game.Game()`. Also, If you want to see the performance of your Minimax or AlphaBeta agents, you may need to add their classes as a python file to the `src` folder and import their classes in the cell below. 

In [5]:
%%python
# Change the above line to %%python if youre using Windows OS
from src import game
from src.player import Player
from src.random_greedy_player import RandomGreedyPlayer
from src.random_player import RandomPlayer

random_greedy_game = game.Game(RandomPlayer, RandomGreedyPlayer)
print(random_greedy_game.play(True))

{0: -178.0, 1: 220.0}



## Minimax Agent (30 Points)


To gain insights into the game, start by reading the `board.py`, `game.py`, and `player.py` files to understand the game implementation. Then, implement a minimax agent. Recall that the minimax algorithm evaluates game states and selects optimal moves. Compare the performance of the random greedy and minimax agents to gain insights.

**Note:** To implement the Minimax agent, you should only modify the `MinimaxPlayer` class in the cell below. You can either use the Board class `get_scores` function or define your own score funcion here and use it for the evaluation of a board state.

**Note:** Your minimax implementation should have a depth instance which determines the level to which the algorithm should be executed for each move. It controls the extent of the search tree explored by the algorithm.

**Note:** Feel free to add cells if you need to, but don't erase the existing cells. 

### Implementation (25 Points)

In [5]:
class MinimaxPlayer(Player):
    def __init__(self, player_number, board, depth=3):
        super().__init__(player_number, board)
        self.depth = depth

    def get_next_move(self):
        # TODO: Implement this function based on the minimax algorithm
        self.board.start_imagination()
        board = self.board.get_imaginary_board()
        move, score = self.maximize(board)
        if not self.board.is_move_valid(self.board.get_board_grid(), move, self.player_number):
            raise Exception("Invalid move")
        return move

    def get_range(self, pn):
        range_n = 0, self.board.get_n()
        step = 1
        if pn == 0:
            range_n = self.board.get_n() - 1, -1
            step = -1
        return range_n[0], range_n[1], step

    def maximize(self, state, cur_depth=1):
        if cur_depth == self.depth:
            return self.best_move(state)

        actions = []
        r = self.get_range(self.player_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.player_number:
                    for direction in self.board.get_possible_directions(self.player_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(state)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.player_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.player_number)
                            if game_over == self.player_number:
                                scores = {self.opponent_number: float('-inf'), self.player_number: float('inf')}
                                actions.append((move, scores))
                            elif game_over == self.opponent_number:
                                scores = {self.player_number: float('-inf'), self.opponent_number: float('inf')}
                                actions.append((move, scores))
                            else:
                                actions.append((move, self.minimize(self.board.imaginary_board_grid, cur_depth + 1)[1]))

        action = max(actions, key=lambda x: x[1][self.player_number])
        return action
    
    
    def minimize(self, state, cur_depth=1):
        if cur_depth == self.depth:
            return self.best_move_op(state)

        actions = []
        r = self.get_range(self.opponent_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.opponent_number:
                    for direction in self.board.get_possible_directions(self.opponent_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(state)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.opponent_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.opponent_number)
                            if game_over == self.player_number:
                                scores = {self.opponent_number: float('-inf'), self.player_number: float('inf')}
                                actions.append((move, scores))
                            elif game_over == self.opponent_number:
                                scores = {self.player_number: float('-inf'), self.opponent_number: float('inf')}
                                actions.append((move, scores))
                            else:
                                actions.append((move, self.maximize(self.board.get_imaginary_board(), cur_depth + 1)[1]))

        action = max(actions, key=lambda x: x[1][self.opponent_number])
        return action

    def best_move(self, board):
        max_score = -float('inf')
        best_scores = None
        best_move = None
        r = self.get_range(self.player_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.player_number:
                    for direction in self.board.get_possible_directions(self.player_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(board)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.player_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.player_number)
                            if scores[self.player_number] > max_score:
                                max_score = scores[self.player_number]
                                best_scores = scores
                                best_move = move
        return best_move, best_scores


    def best_move_op(self, board):
        max_score = -float('inf')
        best_scores = None
        best_move = None
        r = self.get_range(self.opponent_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.opponent_number:
                    for direction in self.board.get_possible_directions(self.opponent_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(board)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.opponent_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.opponent_number)
                            if scores[self.opponent_number] > max_score:
                                max_score = scores[self.opponent_number]
                                best_scores = scores
                                best_move = move
        return best_move, best_scores

### Time Analysis (5 Points)

Now play the game 5 times between two random players and calculate the average execution time. Also do this for the game of a random player and a minimax player. 

In [7]:
# TODO: calculate and print the average execution time of two random players
mean_time_random = 0
n = 5
for i in range(n):
    time_start = time.time()
    test = game.Game(RandomPlayer, RandomPlayer)
    test.play()
    total_time = time.time() - time_start
    mean_time_random += total_time

mean_time_random /= n

print("Random Players Average Execution Time:", mean_time_random, "seconds")

# TODO: calculate and print the average execution time of a random player and a minimax players with depth=3 
mean_time_minimax = 0
for i in range(n):
    time_start = time.time()
    test = game.Game(MinimaxPlayer, RandomPlayer)
    test.play(False)
    total_time = time.time() - time_start
    mean_time_minimax += total_time

mean_time_minimax /= n
print("Minimax Player and Random Player Average Execution Time:", mean_time_minimax, "seconds")

Random Players Average Execution Time: 0.003999137878417968 seconds
Minimax Player and Random Player Average Execution Time: 1.4065239906311036 seconds


## Alphabeta Agent (30 Points)


As we can see from the above code, the Minimax algorithm takes much longer time to execute. To improve the execution time, we can implement the AlphaBeta pruning algorithm. In the next cell, we will be implementing the AlphaBeta player.


### Implementation (25 Points)

In [8]:
class AlphaBetaPlayer(Player):
    def __init__(self, player_number, board, depth=3):
        super().__init__(player_number, board)
        self.depth = depth

    def get_next_move(self):
        self.board.start_imagination()
        board = self.board.get_imaginary_board()
        move, score = self.maximize(board, float('-inf'), float('inf'))
        if not self.board.is_move_valid(self.board.get_board_grid(), move, self.player_number):
            raise Exception("Invalid move")
        return move

    def get_range(self, pn):
        if pn == 0:
            return self.board.get_n() - 1, -1, -1
        else:
            return 0, self.board.get_n(), 1

    def maximize(self, state, alpha, beta, cur_depth=1):
        if cur_depth == self.depth:
            return self.best_move(state)

        actions = []
        r = self.get_range(self.player_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.player_number:
                    for direction in self.board.get_possible_directions(self.player_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(state)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.player_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.player_number)
                            if game_over == self.player_number:
                                scores = {self.opponent_number: float('-inf'), self.player_number: float('inf')}
                                actions.append((move, scores))
                            elif game_over == self.opponent_number:
                                scores = {self.player_number: float('-inf'), self.opponent_number: float('inf')}
                                actions.append((move, scores))
                            else:
                                minimize = self.minimize(self.board.get_imaginary_board(), alpha, beta, cur_depth + 1)
                                actions.append((move, minimize[1]))
                            minimize = actions[-1]
                            if minimize[1][self.player_number] >= beta:
                                return minimize
                            alpha = max(alpha, minimize[1][self.player_number])

        action = max(actions, key=lambda x: x[1][self.player_number])
        return action
    
    def minimize(self, state, alpha, beta, cur_depth=1):
        if cur_depth == self.depth:
            return self.best_move_op(state)

        actions = []
        r = self.get_range(self.opponent_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.opponent_number:
                    for direction in self.board.get_possible_directions(self.opponent_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(state)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.opponent_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.opponent_number)
                            if game_over == self.player_number:
                                scores = {self.opponent_number: float('-inf'), self.player_number: float('inf')}
                                actions.append((move, scores))
                            elif game_over == self.opponent_number:
                                scores = {self.player_number: float('-inf'), self.opponent_number: float('inf')}
                                actions.append((move, scores))
                            else:
                                maximize = self.maximize(self.board.get_imaginary_board(), alpha, beta, cur_depth + 1)
                                actions.append((move, maximize[1]))
                            maximize = actions[-1]
                            if maximize[1][self.opponent_number] <= alpha:
                                return maximize
                            beta = min(beta, maximize[1][self.opponent_number])

        action = max(actions, key=lambda x: x[1][self.opponent_number])
        return action

    def best_move(self, board):
        max_score = -float('inf')
        best_scores = None
        best_move = None
        r = self.get_range(self.player_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.player_number:
                    for direction in self.board.get_possible_directions(self.player_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(board)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.player_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.player_number)
                            if scores[self.player_number] > max_score:
                                max_score = scores[self.player_number]
                                best_scores = scores
                                best_move = move
        return best_move, best_scores

    def best_move_op(self, board):
        max_score = -float('inf')
        best_scores = None
        best_move = None
        r = self.get_range(self.opponent_number)
        for i in range(r[0], r[1], r[2]):
            for j in range(r[0], r[1], r[2]):
                if self.board.get_board_grid()[i][j] == self.opponent_number:
                    for direction in self.board.get_possible_directions(self.opponent_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        self.board.imaginary_board_grid = np.copy(board)
                        if self.board.is_move_valid(self.board.get_imaginary_board(), move, self.opponent_number):
                            scores, game_over = self.board.place_piece_imaginary(move, self.opponent_number)
                            if scores[self.opponent_number] > max_score:
                                max_score = scores[self.opponent_number]
                                best_scores = scores
                                best_move = move
        return best_move, best_scores


### Time Analysis (5 Points)

Now play the game 5 times between the random player and an alphabeta player and calculate the average execution time. 

In [9]:
# TODO: calculate and print the average execution time of a random player and an alpha-beta players with depth=3 
mean_time_alphabeta = 0
n = 5
for i in range(n):
    time_start = time.time()
    test = game.Game(AlphaBetaPlayer, RandomPlayer)
    test.play(False)
    total_time = time.time() - time_start
    mean_time_alphabeta += total_time

mean_time_alphabeta /= n

print("Alphabeta Player and Random Player Average Execution Time:", mean_time_alphabeta, "seconds")

Alphabeta Player and Random Player Average Execution Time: 0.441999626159668 seconds


##### **Note:** The alphabeta agent should be approximately 5X-10X faster than the minimax player.


## Win Probability Analysis (35 Points)

In this section, We will simulate 50 games for your AlphaBeta player against other players and analyze their win rates. Additionally, we will have AlphaBeta players with different depths play against each other. 

Assume you are always the second player and the black player will always start first. 

If the agent is implemented correctly, with a depth of two or more it should win all the mentioned agents with a win rate > 0.7.

**Note:** You can use the `bulk_play` function from `game.py` for this purpose.


In [10]:
import pandas as pd

depths = [1, 2, 3]  # List of different second_player_depth values

In [11]:
def compute_results_different_depths(first_player_class, depths, first_player_depth=None, n=50):
    results = []
    sample_game = game.Game(first_player_class, AlphaBetaPlayer)

    for depth in depths:
        if first_player_depth is not None:
            result = sample_game.bulk_play(n, first_player_depth=first_player_depth, second_player_depth=depth)
        else:
            result = sample_game.bulk_play(n, second_player_depth=depth)
        win_rate = result['white'] / n  # Calculate win rate
        results.append({'Depth': depth, 'Win Rate': win_rate})

    df = pd.DataFrame(results)
    return df

### Random Player

In [12]:
results = compute_results_different_depths(RandomPlayer, depths)
print(results)

   Depth  Win Rate
0      1       1.0
1      2       1.0
2      3       1.0


### Random Greedy Player

In [13]:
results = compute_results_different_depths(RandomGreedyPlayer, depths)
print(results)

   Depth  Win Rate
0      1      0.98
1      2      1.00
2      3      1.00


### Greedy Player

In [14]:
results = compute_results_different_depths(Player, depths)
print(results)

   Depth  Win Rate
0      1       0.0
1      2       1.0
2      3       1.0


### Alphabeta Player with depth 2

**Note:** In this part each game may take up to ~45 seconds.

In [15]:
results = compute_results_different_depths(AlphaBetaPlayer, [2, 3, 4], first_player_depth=2, n=30)
print(results)

   Depth  Win Rate
0      2       1.0
1      3       1.0
2      4       1.0



## Conclusion (5 Points)

#### Based on the results of the AlphaBeta player with other players, what is your conclusion?


**Answer:** AlphaBeta not only achieves a closer play time to greedy and random player, it also is superior to them and with using more time to calculate it can preform better